In [3]:
from ortools.linear_solver import pywraplp
import networkx as nx

In [22]:



# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
# x and y are integer non-negative variables.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Number of variables =', solver.NumVariables())

# x + 7 * y <= 17.5.
p = x 
p += 7 * y
solver.Add(p <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())




Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 8.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


In [13]:
class MRPP_2_IP:

    def __init__(self, graph, num_agents):

        self.num_agents = num_agents    
        self.graph_original = graph
        for e in self.graph_original.edges():
            self.graph_original[e[0]][e[1]]['length'] = int(self.graph_original[e[0]][e[1]]['length'])
        self.graph_complete = graph.copy()
        self.nodes = list(self.graph_original.nodes())
        self.paths = {}
        # self.dist_from_pn = {}
        # self.dist_to_pn = {}
        # self.remaining_nodes = self.nodes.copy()
        # self.time_period = 0
        # for n in self.priority_nodes:
        #     self.remaining_nodes.remove(n)
        #     self.dist_to_pn[n] = {}
        #     self.dist_from_pn[n] = {}
        # print('1')
        paths = dict(nx.all_pairs_dijkstra_path(self.graph_original, weight = 'length'))
        # print('2')
        for i in self.nodes:
            # f = False
            # if i in self.priority_nodes:
            #     f = True
            for j in self.nodes:
                # t = False
                # if j in self.priority_nodes:
                #     t = True
                if i == j and not (i, j) in self.graph_complete.edges():
                    self.graph_complete.add_edge(i, j)
                    self.graph_complete[i][j]['name'] = '{}to{}'.format(i, j)
                    self.graph_complete[i][j]['length'] = 0
                    self.paths[(i, j)] = [i, j]
                elif i != j and not (i, j) in self.graph_complete.edges():
                    self.graph_complete.add_edge(i, j)
                    self.graph_complete[i][j]['name'] = '{}to{}'.format(i, j)
                    self.graph_complete[i][j]['length'] = 0
                    self.paths[(i, j)] = paths[i][j]
                    for k in range(len(self.paths[i, j]) - 1):
                        self.graph_complete[i][j]['length'] += self.graph_original[paths[i][j][k]][paths[i][j][k + 1]]['length']
                elif i != j:
                    self.paths[(i, j)] = [i, j]

        self.edges = list(self.graph_complete.edges())
        self.solver = pywraplp.Solver.CreateSolver('SCIP')
        self.E = len(self.edges)
        self.N = len(self.nodes)
        
       
        self.x = {}
        for a in range(self.num_agents):
            self.x[a] = {}
            for i, j in enumerate(self.edges):
                self.x[a][j] = self.solver.IntVar(0, 1, 'x[{}, {}]'.format(j, a))
        
        self.a = {}
        for a in range(self.num_agents):
            self.a[a] = {}
            for i, j in enumerate(self.nodes):
                self.a[a][j] = self.solver.IntVar(0, 1, 'a[{}, {}]'.format(j, a))

        self.b = {}
        for a in range(self.num_agents):
            self.b[a] = {}
            for i, j in enumerate(self.nodes):
                self.b[a][j] = self.solver.IntVar(0, 1, 'b[{}, {}]'.format(j, a))
        

In [18]:
g = nx.read_graphml('./graphs/grid_5_5.graphml')
test = MRPP_2_IP(g, 4)
g1 = test.graph_complete

In [16]:
print(test.N, test.E, test.solver.NumVariables())

25 625 2700


In [23]:
list(g1.in_edges('0'))

[('1', '0'),
 ('5', '0'),
 ('0', '0'),
 ('2', '0'),
 ('3', '0'),
 ('4', '0'),
 ('6', '0'),
 ('7', '0'),
 ('8', '0'),
 ('9', '0'),
 ('10', '0'),
 ('11', '0'),
 ('12', '0'),
 ('13', '0'),
 ('14', '0'),
 ('15', '0'),
 ('16', '0'),
 ('17', '0'),
 ('18', '0'),
 ('19', '0'),
 ('20', '0'),
 ('21', '0'),
 ('22', '0'),
 ('23', '0'),
 ('24', '0')]